In [9]:
import os
import time

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, SequentialSampler, ConcatDataset

from chord_recognition.dataset import AudioDataset
from chord_recognition.metrics import compute_eval_measures
from chord_recognition.cnn import model
from chord_recognition.predict import predict_annotations, model, device

In [10]:
# chord_dataset = AudioDataset(audio_dir="data/robbie_williams/mp3/",
#                              ann_dir="data/robbie_williams/chordlabs/",
#                             window_size=8192, hop_length=4096)
chord_dataset = AudioDataset(audio_dir="data/queen/mp3/",
                             ann_dir="data/queen/chordlabs/",
                             window_size=8192, hop_length=4096)
queen = AudioDataset(audio_dir="data/beatles/mp3s-32k/",
                     ann_dir="data/beatles/chordlabs/",
                     window_size=8192, hop_length=4096)
chord_dataset = ConcatDataset([chord_dataset, queen])
loader_chord = DataLoader(chord_dataset, batch_size=None, num_workers=0)
sampler = SequentialSampler(chord_dataset)

In [12]:
def estimate_chords(dataloader, model, device, batch_size=32,
                    drop_last=False):
    model.eval()  # set model to evaluation mode
    total_R = total_P = total_F = 0.0
    count = 0
    
    for idx in dataloader:
        sample = dataloader.data_source[idx]
        sample_name = sample['sample']
        chromagram = sample['chromagram']
        
        result = predict_annotations(chromagram, model, device, batch_size=batch_size)
        ann_matrix = sample['ann_matrix']
        count += 1
        
        P, R, F1, TP, FP, FN = compute_eval_measures(ann_matrix, result)
        total_R += R
        total_P += P
        total_F += F1
        title = f'Eval: (N=%d, TP=%d, FP=%d, FN=%d, P=%.3f, R=%.3f, F=%.3f) - {sample_name}' % (result.shape[1], TP, FP, FN, P,R,F1)
        print(title)
    total_R /= count
    total_P /= count
    total_F /= count
    print(f'Total R: {total_R}')
    print(f'Total P: {total_P}')
    print(f'Total F: {total_F}')

In [13]:
estimate_chords(sampler, model, device, batch_size=8)

Eval: (N=3856, TP=2647, FP=1209, FN=33, P=0.686, R=0.988, F=0.810) - Greatest_Hits_I/01 Bohemian Rhapsody
Eval: (N=2337, TP=2113, FP=224, FN=24, P=0.904, R=0.989, F=0.945) - Greatest_Hits_I/02 Another One Bites The Dust
Eval: (N=2200, TP=2080, FP=120, FN=34, P=0.945, R=0.984, F=0.964) - Greatest_Hits_I/04 Fat Bottomed Girls
Eval: (N=1979, TP=1522, FP=457, FN=20, P=0.769, R=0.987, F=0.865) - Greatest_Hits_I/05 Bicycle Race
Eval: (N=1855, TP=1234, FP=621, FN=19, P=0.665, R=0.985, F=0.794) - Greatest_Hits_I/06 You're My Best Friend
Eval: (N=2280, TP=1266, FP=1014, FN=22, P=0.555, R=0.983, F=0.710) - Greatest_Hits_I/07 Don't Stop Me Now
Eval: (N=2466, TP=1933, FP=533, FN=25, P=0.784, R=0.987, F=0.874) - Greatest_Hits_I/08 Save Me
Eval: (N=1760, TP=1717, FP=43, FN=43, P=0.976, R=0.976, F=0.976) - Greatest_Hits_I/09 Crazy Little Thing Called Love
Eval: (N=3205, TP=2022, FP=1183, FN=9, P=0.631, R=0.996, F=0.772) - Greatest_Hits_I/10 Somebody To Love
Eval: (N=1894, TP=1384, FP=510, FN=32, P=0.

Eval: (N=197, TP=144, FP=53, FN=6, P=0.731, R=0.960, F=0.830) - Let_It_Be/05-Dig_It
Eval: (N=949, TP=827, FP=122, FN=15, P=0.871, R=0.982, F=0.924) - Let_It_Be/06-Let_It_Be
Eval: (N=171, TP=157, FP=14, FN=8, P=0.918, R=0.952, F=0.935) - Let_It_Be/07-Maggie_Mae
Eval: (N=861, TP=522, FP=339, FN=21, P=0.606, R=0.961, F=0.744) - Let_It_Be/08-I_ve_Got_A_Feeling
Eval: (N=686, TP=655, FP=31, FN=10, P=0.955, R=0.985, F=0.970) - Let_It_Be/09-One_After_909
Eval: (N=852, TP=601, FP=251, FN=13, P=0.705, R=0.979, F=0.820) - Let_It_Be/10-The_Long_And_Winding_Road
Eval: (N=609, TP=391, FP=218, FN=14, P=0.642, R=0.965, F=0.771) - Let_It_Be/11-For_You_Blue
Eval: (N=757, TP=545, FP=212, FN=6, P=0.720, R=0.989, F=0.833) - Let_It_Be/12-Get_Back
Eval: (N=678, TP=548, FP=130, FN=16, P=0.808, R=0.972, F=0.882) - Magical_Mystery_Tour/01-Magical_Mystery_Tour
Eval: (N=709, TP=258, FP=451, FN=1, P=0.364, R=0.996, F=0.533) - Magical_Mystery_Tour/02-The_Fool_On_The_Hill
Eval: (N=537, TP=400, FP=137, FN=12, P=0.745

Eval: (N=758, TP=683, FP=75, FN=36, P=0.901, R=0.950, F=0.925) - The_White_Album_Disc_1/06-The_Continuing_Story_Of_Bungalow_Bill
Eval: (N=1107, TP=818, FP=289, FN=28, P=0.739, R=0.967, F=0.838) - The_White_Album_Disc_1/07-While_My_Guitar_Gently_Weeps
Eval: (N=640, TP=494, FP=146, FN=10, P=0.772, R=0.980, F=0.864) - The_White_Album_Disc_1/08-Happiness_Is_A_Warm_Gun
Eval: (N=589, TP=225, FP=364, FN=5, P=0.382, R=0.978, F=0.549) - The_White_Album_Disc_1/09-Martha_My_Dear
Eval: (N=488, TP=444, FP=44, FN=18, P=0.910, R=0.961, F=0.935) - The_White_Album_Disc_1/10-I_m_So_Tired
Eval: (N=542, TP=293, FP=249, FN=20, P=0.541, R=0.936, F=0.685) - The_White_Album_Disc_1/11-Blackbird
Eval: (N=485, TP=441, FP=44, FN=13, P=0.909, R=0.971, F=0.939) - The_White_Album_Disc_1/12-Piggies
Eval: (N=836, TP=227, FP=609, FN=1, P=0.272, R=0.996, F=0.427) - The_White_Album_Disc_1/13-Rocky_Raccoon
Eval: (N=902, TP=872, FP=30, FN=3, P=0.967, R=0.997, F=0.981) - The_White_Album_Disc_1/14-Don_t_Pass_Me_By
Eval: (N=3